# Kurs 4

## Multiple dispatch

Wie vermutlich schon so ein bisschen angeklungen ist, unterscheidet sich Julia sowohl von statisch-typisierten Programmiersprachen wie C++, als auch von klassischen dynamisch-typisierten Programmiersprachen wie Python.

Bei C++ muss zur Kompilierzeit bekannt sein, was für ein Typ in eine bestimme Funktion gesteckt wird. Dieser Code wird dann in Maschinencode übersetzt. Bei Python muss der Typ einer Variablen wiederum erst zur Laufzeit bekannt sein, allerdings findet auch keine Übersetzung in Maschinencode statt. Julia ist quasi eine Hybrid-Lösung: Sobald eine Zeile Code ausgeführt wird, wird für den *konkret hineingesteckten Typ* ein Kompiliervorgang gestartet und der kompilierte Code ausgeführt. Diesen Vorgang nennt man *multiple dispatch*, wozu Julia einen *just-in-time compiler* (JIT) nutzt. Dadurch ist Julia faktisch dynamisch-typisiert, kann aber bei richtiger Verwendung quasi genauso schnell wie C/C++ sein.

### Beispiel

Wir vergleichen, was nach dem Kompilierprozess rauskommt, wenn man einmal ```Integer``` und einmal ```String``` in dieselbe Funktion wirft:

In [ ]:
# Beispielfunktion für unterschiedliche Input-Typen
f(a, b) = a * b
# Wir nutzen ein Makro (lernen wir später kennen)
@code_llvm f(1, 2)

In [ ]:
@code_llvm f("1", "2")

Dass hier zwei verschiedene Outputs herauskommen, ist im Wesentlichen der Grund dafür, dass Julia performant (schnell) ist. Allerdings kann genau dieses Verhalten manchmal auch für langsamen Code führen – nämlich genau dann, wenn der Output Kompilierprozesses nicht spezialisiert genug ist (mehr dazu später).

## Pakete

Für die kommenden Anwendungen werden wir sogenannte Pakete (packages) nutzen, die uns zusätzliche Funktionalität liefern. Wenn wir ein Paket namens ```ExamplePackage``` installieren wollen, dann geht das mittels ```using Pkg``` und ```Pkg.add("ExamplePackage")```.[^1]

In der Julia-Konsole ist das durch den Paketmanager, welchen man durch die Eingabe von ```]``` erreicht, sogar noch einfacher (```add ExamplePackage```). Ein installiertes Paket lässt sich dann mit ```using ExamplePackage``` in Julia nutzen.

[^1]:
    Tatsächlich laden wir hier mit ```using``` ein Modul bzw. namespace (wir wissen noch nicht was das ist), generell sind Pakete hauptsächlich Modulsammlungen.

## Pipes

Durch Pipes bekommen wir eine hübschere Syntax:

In [ ]:
# Beispielrechnung mit verschachtelten Funktionen
foo = 0; add6(x) = x + 6; mul2(x) = 2*x
println(mul2(add6(foo)))

In [ ]:
# Base Julia hat schon Pipes
foo |> add6 |> mul2 |> println

Das Paket Pipe.jl bietet uns noch ein paar zusätzliche Möglichkeiten:

In [ ]:
# Die Ausgabe eines solchen Befehls ist beim ersten Ausführen länger
using Pkg; Pkg.add("Pipe")

In [ ]:
# lade Paket
using Pipe

# Beispielfunktion mit zwei Argumenten
polynom(x, y) = x^2 + y + 5

# benutze den @pipe decorator
@pipe foo |> polynom(_, 2)

## DataFrames

Für unser nächstes Thema brauchen wir das Paket [DataFrames.jl](https://dataframes.juliadata.org/stable/).

In [ ]:
Pkg.add("DataFrames")

In [ ]:
using DataFrames

DataFrames sind im Grunde genommen Tabellen, wie zum Beispiel in Excel. Wer schon ```dplyr``` oder ```pandas``` kennt, findet [hier](https://dataframes.juliadata.org/stable/man/comparisons/) einen guten Vergleich zu ```DataFrames.jl```.

In unserem ersten Beispiel schauen wir uns einen Datensatz mit vergebenen Noten an:

In [ ]:
function grades_2020()
    name = ["Sally", "Bob", "Alice", "Hank"]
    grade_2020 = [1, 5, 8.5, 4]
    DataFrame(; name, grade_2020)
end
df = grades_2020()

Auf Spalten kann direkt über ihren Namen zugegriffen werden:

In [ ]:
df.name 

Alternativ würde auch die Syntax ```df[!, :name]``` funktionieren.
Hierbei ist wichtig zu verstehen, dass dies keine Kopie der Spalte erzeugt, sondern Änderungen sich direkt in den DataFrame übertragen. 

In [ ]:
df.name[1] = "Sharon"
df

```df[:, :name_der_spalte]``` oder ```df[:, nummer_der_spalte]``` hingegen erzeugen Kopien der ausgewählten Spalten des DataFrames (das ist also anders als bei Arrays):

In [ ]:
names = df[!, :name]
names[1] = "Alice" # oder df[:,1]

In [ ]:
names[1] = "Alice"
df

#### ```filter```

Die Funktion ```filter``` hilft Zeilen eines DataFrames nach beliebigen Kriterien auszuwählen, als Beispiel hierfür definieren wir uns zunächst folgende Funktion:

In [ ]:
equals_alice(name::String) = name == "Alice"
equals_alice("Bob")

Mithilfe von ```filter``` können wir jetzt alle Einträge der Spalte ```name``` in dem Dataframe durchgehen und die Reihen mit dem Namen ```"Alice"``` filtern.


In [ ]:
filter(:name => equals_alice, df)

Alternativ hätten wir uns auch die Hilfsfunktion ```equals_alice``` sparen und stattdessen eine anonyme Funktion nutzen können: 

In [ ]:
filter(:name => x -> x == "Bob", df)

Noch kürzer geht es mithilfe der generischen Funktion ```==("Bob")```:

In [ ]:
filter(:name => ==("Bob"), df)

#### ```subset``` und ```select```

Statt ```filter``` können (und wollen wir meistens) auch die Funktion ```subset``` nutzen. Aus zwei Gründen sollte man aber ```filter``` trotzdem schon mal gesehen haben:

- ```filter``` funktioniert auch für andere Typen als DataFrames (zum Beispiel Dicts)
- zum Teil ist ```filter``` performanter

Umgekehrt ist ```subset``` aber zumeist ein bisschen angenehmer:

- besseres handling von fehlenden Werten
- Syntax ist konsistent mit den anderen Befehlen für DataFrames.

Zentral ist jedenfalls, dass ```subset``` nicht Werte einzelner Reihen vergleicht, sondern stets die komplette Spalte vergleicht:

In [ ]:
# vergleiche mit filter, unsere anonyme Funktion operiert nun auf einen Vektor!
subset(df, :name => x -> x .==("Bob"))

Wenn wir unser Problem wie gehabt nicht in einen Vergleich der ganzen Spalte umformulieren wollen, dann geht das mithilfe der Funktion ```ByRow```:

In [ ]:
subset(df, :name => ByRow(==("Bob")))

Die Funktion ```select``` filtert im Gegensatz zu ```filter```/```subset``` nicht Reihen, sondern Spalten aus.

In [ ]:
function responses()
    id = [1, 2]
    q1 = [28, 61]
    q2 = [:us, :fr]
    q3 = ["F", "B"]
    q4 = ["B", "C"]
    q5 = ["A", "E"]
    DataFrame(; id, q1, q2, q3, q4, q5)
end
df2 = responses()

Beispielsweise können wir so die Spalten ```id``` und ```q1``` auswählen:

In [ ]:
select(df2, :id, :q1) # alternativ würde auch select(df2, "id", "q1") funktionieren 

Mithilfe der Funktion ```Not``` kann man eine oder mehrere Spalten aussortieren:

In [ ]:
select(df2, Not([:q2, :q5]))

Die Position von Spalten kann derart verändert werden:

In [ ]:
select(df2, :q3, :) # erst q3, dann der Rest

In [ ]:
select(df2, 2, :q3, :) # erst die zweite Spalte (:q1), dann :q3, dann der Rest

Und zuletzt können mithilfe von ```select``` auch die Namen der Spalten geändert werden:

In [ ]:
select(df2, 1 => "participant", :q1 => "age", :q2 => "nationality")
df2

Wie für Julia typisch, gibt es alle oben genanten Funktionen auch mit einem ```!``` (wie etwa ```select!```).
Der Unterschied besteht darin, dass in diesem Fall keine Kopie des Dataframes erstellt, sondern der orginale DataFrame verändert wird.
 

### Datentypen und kategorische Variablen

Wie man in den oberen Bespielen erkennen kann, versucht Julia jeder Spalte einen Datentyp zuzordnen, was allerdings nicht immer ganz so gut funktioniert.

In [ ]:
function wrong_types()
    id = 1:4
    date = ["28-01-2018", "03-04-2019", "01-08-2018", "22-11-2020"]
    age = ["adolescent", "adult", "infant", "adult"]
    DataFrame(; id, date, age)
end
df = wrong_types()

Falsche Datentypen können das Sortieren in DataFrames erschweren. In dem oberen Fall hat zum Beispiel die Spalte ```date``` das Format ```String```, obwohl es in Julia dafür einen speziellen Datentyp (nämlich ```Date```) gibt, mithilfe dessen man Daten im Datumsformat einfach vergleichen kann. Mit dem Paket ```Dates``` lässt sich dies aber leicht beheben.

In [ ]:
Pkg.add("Dates")

In [ ]:
using Dates

function fix_date_format(df)
    dates = Date.(df.date, dateformat"dd-mm-yyyy") # specify date format
    df.date = dates # reassign dates
end

fix_date_format(df)
df

Zur Überprüfung vergleichen wir das Geburtsdatum der Personen 1 und 2 und erhalten: 

In [ ]:
df[1, :date] < df[2, :date]

In der Spalte ```age``` trifft man auf ein ähnliches Problem, da man den Variablen gerne eine hierarchische Struktur geben würde: ```adult > adolescent > infant```. Dies lässt sich mithilfe des Paketes ```CategoricalArrays``` beheben:

In [ ]:
Pkg.add("CategoricalArrays")

In [ ]:
using CategoricalArrays

function fix_age(df)
    levels = ["infant", "adolescent", "adult"]
    ages = categorical(df.age; levels, ordered = true)
    df.age = ages
end

fix_age(df)
df

Auch hier kann man noch einmal überprüfen ob sich die Personen anhand ihres Alter vergleichen lassen können : 

In [ ]:
df[1, :age] < df[2, :age]

### Datensätze zusammenführen

Im folgenden Abschnitt wird behandelt, wie man verschiedene DataFrames zusammenführen bzw. kombinieren kann. Hierfür nimmt man meistens verschiedene Versionen der Funktion ```join```; die jeweilige Funktionalität ist im [Cheatsheet](https://ahsmart.com/assets/pages/data-wrangling-with-data-frames-jl-cheat-sheet/DataFramesCheatSheet_v1.x_rev1.pdf) von Tom Kwong schön visualisiert.

Betrachten wir zwei DataFrames:

In [ ]:
function grades_2021() # grades_2020() haben wir schon
    name = ["Kevin", "Sally", "Hank"]
    grade_2021 = [8, 7, 5.5]
    DataFrame(; name, grade_2021)
end
grades_2021()

In [ ]:
grades_2020()

Bei ```innerjoin``` wird ein Argument/Spaltenname mitgegeben, über welchen die beiden DataFrames zusammen geführt werden sollen. So werden beispielsweise für ```:name``` alle Elemente aus der Spalte ```:name``` des einen DataFrames mit den Elementen des anderen DataFrames verglichen. Falls diese Einträge übereinstimmen, werden die restliche Information (Einträge der Reihe) aus beiden DataFrames in Spalten zusammengeführt.

In [ ]:
innerjoin(grades_2020(), grades_2021(), on=:name)

```outerjoin``` nimmt hingegen alle Elemente, die in den jeweiligen DataFrames zumindest einmal vorkommen und führt diese zusammen. Hierbei werden nicht vorhandene Information durch ein "missing" ersetzt.  

In [ ]:
outerjoin(grades_2020(), grades_2021(); on=:name)

Zuletzt übergibt ```leftjoin```/```rightjoin``` alle Werte aus dem linken/rechten Dataframe und führt sie mit den Einträgen aus dem rechten/linken DataFrame zusammen, für welche der rechte/linke Dataframe in der entsprechenden Kategorie (zum Beispiel ```:name```) übereinstimmt:

In [ ]:
leftjoin(grades_2020(), grades_2021(); on=:name)